# Llama 2 Fine-Tuning Evaluation

**This notebook is incomplete**

In [ ]:
import os
import pandas as pd

base_dir = os.path.join('./praktikum-ise-2023-patrick-zierahn')
data_dir = os.path.join(base_dir, 'data')

reports_dir = os.path.join(base_dir, 'reports', 'llama2_fine_tune')
os.makedirs(reports_dir, exist_ok=True)

llama_data_dir = os.path.join(data_dir, 'llama2_fine_tune')

taxonomy_file = os.path.join(data_dir, 'software_architecture', 'taxonomy_explanation.json')

In [ ]:
from src.utils.utils_json import read_json

taxonomy_explanation = read_json(taxonomy_file)

research_objects_all = ""
for obj in taxonomy_explanation["Research Object"].keys():
    research_objects_all += obj + "\n"

In [ ]:
df_train = pd.read_csv(os.path.join(llama_data_dir, 'zealous_chaum_train.csv'))
df_eval = pd.read_csv(os.path.join(llama_data_dir, 'zealous_chaum_test.csv'))

In [ ]:
df_eval.head()

In [ ]:
from typing import List
from src.taxonomy.utils import parse_taxonomy


def get_research_objects(meta: str) -> List[str]:
    tree = parse_taxonomy(meta)
    research_obj = tree.children("Research Object")

    objs = []
    for obj in research_obj:
        objs.append(obj.tag)

    return objs

In [ ]:
prompt = df_train['title'].iloc[0] + '\n' + df_train['abstract'].iloc[0]

In [ ]:
print(f"Prompt: {prompt}")
print(f"Research Object: {get_research_objects(df_train['classes'].iloc[0])}")

In [ ]:
import transformers
import torch

# model = "meta-llama/Llama-2-7b-hf"
# model = "./models/llama-2-7b-zealous_chaum/"
# model = "./models/llama-2-7b-clever_meitner/"
model = "./models/llama-2-7b-optimistic_knuth/"

tokenizer = transformers.AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float32,
    device_map="auto",
)

sequences = pipeline(
    f"Categories the paper into the following research objects:\n{research_objects_all}\n\n"
    f"### Paper: Title:{df_train['title'].iloc[0]}\nAbstract: {df_train['abstract'].iloc[0]}\n"
    f"### Response:",
    # prompt,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=4098,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")